In [1]:

from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import pprint
import allensdk.brain_observatory.stimulus_info as stim_info
import numpy as np
from __future__ import print_function
import h5py
from allensdk.brain_observatory.natural_scenes import NaturalScenes
import os
# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
cwd = os.getcwd()


In [ ]:
# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print("all cre lines: " + str(cre_lines))

targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures))


In [2]:
areas = ['VISpm','VISal','VISl','VISp']
cre_lines = ['Cux2-CreERT2','Rbp4-Cre','Rorb-IRES2-Cre']

# get natural scenes dF/F

In [31]:
#save dff traces for cells in given area
#get cells and save. 
#skip if data already saved. 

areas = ['VISpm','VISal','VISl', 'VISp']
interlength = 7 #in frames, 0.25 secs
stim_length = 7 #frames, 0.25 secs
sweep_length = 2 * interlength + stim_length #in frames, 
cre_lines = ['Cux2-CreERT2','Rbp4-Cre','Rorb-IRES2-Cre']  #same cre lines shared across all areas

for aa in range(len(areas)):
    this_area = areas[aa]
    X_ALL = []
    XT_ALL = []
    Y_ALL = []
    cells_all = []
    total_num_cells = 0
    
    exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.NATURAL_SCENES],
                                     cre_lines=cre_lines)
    for ee in range(len(exps)):
        exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.NATURAL_SCENES],
                                         cre_lines=cre_lines)[ee]
        expData = boc.get_ophys_experiment_data(exps['id'])
        time,dffTraces = expData.get_dff_traces() #get traces . dffTraces is cells by time

        #118 scenes, each scene presented 50 times. there is a scene labeled -1 in stimulus table, must be blank
        stim_table = expData.get_stimulus_table('natural_scenes')
        scenes = expData.get_stimulus_template('natural_scenes')
        sceneIDs = np.unique(stim_table.frame)

        cell_ids = expData.get_cell_specimen_ids()
        numCells = len(cell_ids)
        total_num_cells = numCells + total_num_cells

        nTrials = len(stim_table)
        y = np.empty((nTrials))
        y[:] = np.nan
        trials_per_scene = nTrials/len(sceneIDs)
        XT_exp = np.empty((nTrials,numCells,sweep_length))
        
        for scene in sceneIDs:
            trial_mask = (stim_table.frame == scene)
            this_stim_table = stim_table[trial_mask]
            for t in np.arange(1,trials_per_scene+1):
                trial = t + (scene + 1) * trials_per_scene - 1
                y[trial] = scene + 1
                for cell in np.arange(0,numCells):
                    stim_on = this_stim_table.start.iloc[t-1] 
                    start_sweep = stim_on - interlength #start 0.25 secs (7 frames) before stimulus onset
                    end_sweep = stim_on + stim_length + interlength #end 0.25 secs (14 frames) after stimulus onset
                    trange = np.arange(start_sweep ,end_sweep)
                    XT_exp[trial,cell,:] = dffTraces[cell,trange]

        #concatenate experiments
        if ee == 0:
            XT_ALL = XT_exp
            Y_ALL = y
            cells_all = cell_ids

        if ee > 0:
            XT_ALL = np.hstack((XT_ALL,XT_exp))
            Y_ALL = np.vstack((Y_ALL,y))
            cells_all = np.hstack((cells_all,cell_ids))

    print(this_area + ': %d cells' %(total_num_cells))
    filename = this_area + '_sweeps_data.h5'
    
    #save arrays X_ALL & XT_ALL -make this into function
    with h5py.File(filename,'w') as hf:
        hf.create_dataset('X_matrix_time', data=XT_ALL,chunks=True)
        hf.create_dataset('Y_matrix', data=Y_ALL,chunks=True)
        hf.create_dataset('cell_IDs', data=cells_all,chunks=True)

VISpm: 2157 cells
VISal: 2323 cells


VISl: 2606 cells


VISp: 4189 cells


In [6]:
#save dff traces for cells in given area, for a given cell type
#get cells and save. 

areas = ['VISpm','VISal','VISl', 'VISp']
cre_lines = ['Cux2-CreERT2','Rbp4-Cre','Rorb-IRES2-Cre']

interlength = 7 #in frames, 0.25 secs
stim_length = 7 #frames, 0.25 secs
sweep_length = 2*interlength+stim_length #in frames, 1secs

for aa in range(len(areas)):
    this_area = areas[aa]
    
    for cc in range(len(cre_lines)):
        this_cre_line = cre_lines[cc]
        X_ALL = []
        XT_ALL = []
        Y_ALL = []
        cells_all = []
        total_num_cells = 0
        
        exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.NATURAL_SCENES],
                                         cre_lines=[this_cre_line],)
        num_exps = len(exps)

        for ee in range(num_exps):
            exps = boc.get_ophys_experiments(targeted_structures=[this_area],stimuli=[stim_info.NATURAL_SCENES],
                                             cre_lines=[this_cre_line])[ee]
            expData = boc.get_ophys_experiment_data(exps['id'])
            time,dffTraces = expData.get_dff_traces() #get traces . dffTraces is cells by time

            #118 scenes, each scene presented 50 times. there is a scene labeled -1 in stimulus table, must be blank
            stim_table = expData.get_stimulus_table('natural_scenes')
            scenes = expData.get_stimulus_template('natural_scenes')
            sceneIDs = np.unique(stim_table.frame)
            cell_ids = expData.get_cell_specimen_ids()
            numCells = len(cell_ids)
            total_num_cells = numCells + total_num_cells
            nTrials = len(stim_table)
            y = np.empty((nTrials))
            y[:] = np.nan
            trials_per_scene = nTrials/len(sceneIDs)
            XT_exp = np.empty((nTrials,numCells,sweep_length))

            for scene in sceneIDs:
                trial_mask = (stim_table.frame == scene)
                this_stim_table = stim_table[trial_mask]
                for t in np.arange(1,trials_per_scene+1):
                    trial = t + (scene + 1) * trials_per_scene - 1
                    y[trial] = scene + 1
                    for cell in np.arange(0,numCells):
                        stim_on = this_stim_table.start.iloc[t-1] 
                        start_sweep = stim_on - interlength #start one second (28 frames) before stimulus onset
                        end_sweep = stim_on + stim_length + interlength   #end 
                        trange = np.arange(start_sweep ,end_sweep)
                        XT_exp[trial,cell,:] = dffTraces[cell,trange]

            #concatenate experiments
            if ee == 0:
                XT_ALL = XT_exp
                Y_ALL = y
                cells_all = cell_ids
            if ee > 0:
                XT_ALL = np.hstack((XT_ALL,XT_exp))
                Y_ALL = np.vstack((Y_ALL,y))
                cells_all = np.hstack((cells_all,cell_ids))

        print(this_area + ',  ' + this_cre_line + ': ' + str(total_num_cells) + ' cells')
        filename = this_area +'_'+ this_cre_line + '_sweeps_data.h5'

        #save arrays X_ALL & XT_ALL -make this into function
        with h5py.File(filename,'w') as hf:
            hf.create_dataset('X_matrix_time', data=XT_ALL,chunks=True)
            hf.create_dataset('Y_matrix', data=Y_ALL,chunks=True)
            hf.create_dataset('cell_IDs', data=cells_all,chunks=True)

VISpm,  Cux2-CreERT2: 1646 cells
VISpm,  Rbp4-Cre: 303 cells


VISpm,  Rorb-IRES2-Cre: 208 cells


VISal,  Cux2-CreERT2: 1380 cells


VISal,  Rbp4-Cre: 463 cells


VISal,  Rorb-IRES2-Cre: 480 cells

...


VISl,  Rbp4-Cre: 496 cells


VISl,  Rorb-IRES2-Cre: 530 cells


VISp,  Cux2-CreERT2: 2802 cells


VISp,  Rbp4-Cre: 566 cells


VISp,  Rorb-IRES2-Cre: 821 cells
